In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [20]:
def compute_darboux_frame_for_points(pcd: o3d.geometry.PointCloud, sample_points: o3d.geometry.PointCloud, search_radius: float = 0.02):
    """
    Compute the Darboux frame for each point in pcd using sample_points.

    Parameters
    ----------
    pcd : o3d.geometry.PointCloud
        The point cloud for which to compute the Darboux frame.
    sample_points : o3d.geometry.PointCloud
        The point cloud used for sampling to compute the Darboux frame.
    search_radius : float, optional
        The radius for neighborhood search used in PCA, by default 0.02[m].
    
    Returns
    -------
    darboux_frames : list
        A list of rotation matrices representing the Darboux frame for each point in pcd.
    """

    kdtree = o3d.geometry.KDTreeFlann(pcd)
    points = np.asarray(pcd.points)
    sampled = np.asarray(sample_points.points)
    rotation_matrices = []
    frames = []

    for p in sampled:
        [k, idx, _] = kdtree.search_radius_vector_3d(p, search_radius)
        if k < 3:
            frames.append(None) # Not enough neighbors to compute the frame
            print(f'||| warn: Point {p} has less than 3 neighbors, skipping.')
            continue

        neighbors = points[idx]
        # Compute the covariance matrix(centered)
        centered = neighbors - np.mean(neighbors, axis=0)
        cov = centered.T @ centered

        # Compute the eigenvalues(ascending order) and eigenvectors(normalized) 
        eigvals, eigvecs = np.linalg.eigh(cov)

        # Define the Darboux frame
        normal = eigvecs[:, 0] # Normal (Minimum eigenvalue)
        tangent1 = eigvecs[:, 2] # Principal curvature direction (Maximum eigenvalue)
        tangent2 = eigvecs[:, 1] # The other principal curvature direction (Middle eigenvalue)

        # Ensure normal looks outward from origin (assuming the sensor is at the origin (0, 0, 0))
        if np.dot(normal, p) < 0:
            print(f'normal: \n{normal}')
            print(f'p: \n{p}')
            normal = -normal

        # Ensure right-handed coordinate system
        if np.dot(np.cross(tangent1, tangent2), normal) < 0:
            tangent2 = -tangent2 # return tangent2 to become right-handedness

        # Create the rotation matrix
        rotation_matrix = np.column_stack((tangent1, tangent2, normal)) # x, y, z axes
        T = np.eye(4)
        T[:3, :3] = rotation_matrix
        T[:3, 3] = p # Set the translation to the point p
        frames.append(T)

    return frames

In [21]:
# load point cloud
pcd = o3d.io.read_point_cloud('../data/segmented/remote_controller.ply')
sampled_pcd = o3d.io.read_point_cloud('../data/segmented/remote_controller_random_sampled.ply')
# compute the Darboux frame
frames = compute_darboux_frame_for_points(pcd, sampled_pcd)

normal: 
[0.31897529 0.72834842 0.60643495]
p: 
[-0.07855225  0.00958252 -0.36328125]
normal: 
[-0.1591969   0.38903214  0.9073645 ]
p: 
[-0.07080078 -0.09872437 -0.27075195]
normal: 
[0.20421905 0.69654534 0.68783658]
p: 
[-0.06182861 -0.04647827 -0.30981445]
normal: 
[0.4560927  0.69216099 0.55936805]
p: 
[-0.05418396 -0.0380249  -0.32617188]
normal: 
[0.31246475 0.707759   0.63359528]
p: 
[-0.05911255 -0.03634644 -0.32047526]
normal: 
[0.41825295 0.67890839 0.60344666]
p: 
[-0.04934692 -0.05395508 -0.31030273]
normal: 
[-0.03916336  0.66381157  0.74687377]
p: 
[-0.11026001  0.05247498 -0.39978027]
normal: 
[0.28200776 0.71138211 0.6437446 ]
p: 
[-0.07498169 -0.00244808 -0.35058594]
normal: 
[-0.24663562  0.60561938  0.7565686 ]
p: 
[-0.09960938 -0.00077963 -0.35205078]
normal: 
[0.11102009 0.68591888 0.71915911]
p: 
[-0.09838867  0.03787231 -0.38671875]
normal: 
[-0.2314858   0.65185377  0.72215025]
p: 
[-0.12353516  0.06149292 -0.40917969]
normal: 
[0.44644906 0.69193832 0.56736637

In [22]:
# visualize
visuals = []
for T in frames:
    if T is not None:
        frame_axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01)
        #frame.rotate(R, center=(0, 0, 0))  # Rotate the frame to match the Darboux frame
        frame_axes.transform(T)
        visuals.append(frame_axes)
origin = o3d.geometry.TriangleMesh.create_coordinate_frame(0.05)

o3d.visualization.draw_geometries(visuals+[pcd]+[origin], window_name='Darboux Frame Visualization', mesh_show_back_face=True)